# Toxicology Testing
For this assignment, you will use a chemical dataset to train a neural network to predict human reaction to exposure to certain compounds.

# Import Packages

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import deepchem as dc
from sklearn.metrics import accuracy_score

2024-10-05 14:22:59.533730: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-05 14:23:00.551829: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-10-05 14:23:00.551903: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-10-05 14:23:00.551909: W 

# Set Environment

In [3]:
np.random.seed(456)
tf.random.set_seed(456)

# Load and Prep Data

In [4]:
_, (train, valid, test), _ = dc.molnet.load_tox21()
train_X, train_y, train_w = train.X, train.y, train.w
valid_X, valid_y, valid_w = valid.X, valid.y, valid.w
test_X, test_y, test_w = test.X, test.y, test.w
train_y = train_y[:, 0]
valid_y = valid_y[:, 0]
test_y = test_y[:, 0]
train_w = train_w[:, 0]
valid_w = valid_w[:, 0]
test_w = test_w[:, 0]

[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerator
[14:24:18] DEPRECATION WARNING: please use MorganGenerat

# Define Model Placeholders

In [5]:
d = 1024
n_hidden = 50
learning_rate = .001
n_epochs = 10
batch_size = 100

with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (None, d))
    y = tf.placeholder(tf.float32, (None,))

AttributeError: module 'tensorflow' has no attribute 'placeholder'

# Create Hidden Layer

In [ ]:
with tf.name_scope("hidden-layer"):
    W = tf.Variable(tf.random_normal((d, n_hidden)))
    b = tf.Variable(tf.random_normal((n_hidden,)))
    x_hidden = tf.nn.relu(tf.matmul(x, W) + b)

# Complete Architecture

In [ ]:
with tf.name_scope("output"):
    W = tf.Variable(tf.random_normal((n_hidden, 1)))
    b = tf.Variable(tf.random_normal((1,)))
    y_logit = tf.matmul(x_hidden, W) + b
    # The sigmoid gives the class probability of 1
    y_one_prob = tf.sigmoid(y_logit)
    # Rounding P(y=1) will give the correct prediction.
    y_pred = tf.round(y_one_prob)

with tf.name_scope("loss"):
    # Compute the cross-entropy term for each datapoint
    y_expand = tf.expand_dims(y, 1)
    entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y_expand)
    # Sum all contributions
    l = tf.reduce_sum(entropy)

with tf.name_scope("optim"):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(l)

with tf.name_scope("summaries"):
    tf.summary.scalar("loss", l)
    merged = tf.summary.merge_all()

# Add Dropout to a Hidden Layer

# Define a Hidden Layer with Dropout

# Implement Mini-Batch Training

In [ ]:
train_writer = tf.summary.FileWriter('/tmp/fcnet-tox21',
                                     tf.get_default_graph())
N = train_X.shape[0]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
step = 0

for epoch in range(n_epochs):
    pos = 0

while pos N:

      batch_X = train_X[pos:pos+batch_size]
      batch_y = train_y[pos:pos+batch_size]
      feed_dict = {x: batch_X, y: batch_y}
      _, summary, loss = sess.run([train_op, merged, l], feed_dict = feed_dict)
      print("epoch %d, step %d, loss: %f" % (epoch, step, loss))
      train_writer.add_summary(summary, step)

      step += 1
      pos += batch_size

# Make Predictions
valid_y_pred = sess.run(y_pred, feed_dict={x: valid_X})

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2c4d82d2-b129-4486-baa4-05c502f179ef' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>